# Import json to pandas df, flatten all key-value-pairs to individual columns, drop empty and data-free ("type", "label") columns

In [13]:
import pandas as pd
import json
from flatten_json import flatten
from IPython.display import display, HTML, Markdown
from ydata_profiling import ProfileReport
import great_expectations as ge
from datetime import datetime

#Import a csv-File generated from the JSON through OpenRefine
#df = pd.read_csv('FactGrid2Toolbox-json.csv')
#df = df.dropna(subset=['_ - dc:identifier'])

with open('FactGrid2Toolbox.json', 'r') as f:
   dic = json.load(f)
number_in_set = dic['dc:dataset']['dc:sizeOrDuration']
description = dic['dc:dataset']['dc:description']

dic = dic['gndo:differentiatedPerson']
dic = [flatten(d) for d in dic]
df = pd.DataFrame(dic)
df = df.dropna(axis=1, how='all')
df = df.drop(df.filter(regex='@type$|label$', axis=1).columns, axis=1)
df = df.reindex(sorted(df.columns), axis=1)


df.loc[10,'dc:identifier'] = "test"
df.loc[8,'dc:identifier'] = "https://database.factgrid.de/entity/Q1016"

display(HTML("<style>div.output_scroll { height: 12em; }</style>"))
display(HTML(df.to_html()))

,dc:identifier,gndo:affiliation_0_rdfs:literal,gndo:affiliation_1_rdfs:literal,gndo:affiliation_2_rdfs:literal,gndo:affiliation_3_rdfs:literal,gndo:affiliation_4_rdfs:literal,gndo:affiliation_5_rdfs:literal,gndo:affiliation_6_rdfs:literal,gndo:affiliation_7_rdfs:literal,gndo:affiliation_8_rdfs:literal,gndo:affiliation_9_rdfs:literal,gndo:biographicalOrHistoricalInformation_0_rdfs:literal,gndo:dateOfBirth_schema:date,gndo:dateOfDeath_schema:date,gndo:placeOfBirth_0_rdfs:literal,gndo:placeOfDeath_0_rdfs:literal,gndo:preferredNameOfThePerson_gndo:forename,gndo:preferredNameOfThePerson_gndo:surname,gndo:professionOrOccupation_0_rdfs:literal,gndo:professionOrOccupation_1_rdfs:literal,gndo:professionOrOccupation_2_rdfs:literal,gndo:professionOrOccupation_3_rdfs:literal,gndo:professionOrOccupation_4_rdfs:literal,gndo:professionOrOccupation_5_rdfs:literal,gndo:professionOrOccupation_6_rdfs:literal,gndo:professionOrOccupation_7_rdfs:literal,gndo:professionOrOccupation_8_rdfs:literal,gndo:professionOrOccupation_9_rdfs:literal,gndo:variantNameOfThePerson_0_gndo:surname
0,https://database.factgrid.de/entity/Q100,Areopag (Illuminatenorden),Freimaurerei,Illuminatenorden,"Loge ""St. Theodor vom guten Rat"", München",Minervalkirche Athen [München],NaN,NaN,NaN,NaN,NaN,Universität Ingolstadt|Wilhelmsgymnasium München:Rechtswissenschaften,1752-01-01,1799-01-01,München,None,Franz von Paula Edler von,Berger auf Siebenbrunn,Revisionsrat,Schulkommissions-Assessor,Student der Rechte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,https://database.factgrid.de/entity/Q1001,Freimaurerei,Illuminatenorden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1757-01-01,None,None,None,Johann Nepomuk Christian von,Schaden,Hofrat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,https://database.factgrid.de/entity/Q1004,Freimaurerei,Illuminatenorden,"Loge ""Augusta zu den drei Kronen"", Freising",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1756-01-01,None,None,None,Joseph Leonhard,Schaffner,Gerichtspraktikant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,https://database.factgrid.de/entity/Q1006,Freimaurerei,Illuminatenorden,"Loge ""Joseph zu den drei Helmen""/ ""Wilhelm zu den Drei Helmen"", Wetzlar",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universität Heidelberg:Rechtswissenschaften,1758-01-01,1814-01-01,None,None,Clemens August d J Freiherr,Schall zu Bell,Geheimer Rat,Hofmarschall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,https://database.factgrid.de/entity/Q101,Freimaurerei,Illuminatenorden,"Johannis-Loge ""Günther zum stehenden Löwen""/ ""Günther zur Eintracht"", Rudolstadt","Loge ""Anna Amalia zu den drei Rosen"", Weimar","Loge ""Minerva zu den Drei Palmen"", Leipzig",NaN,NaN,NaN,NaN,NaN,Universität Jena|Universität Leipzig:Rechtswissenschaften,1755-04-05,1825-02-05,Rudolstadt,Rudolstadt,Friedrich Heinrich Christoph,Bergmann,Druckereibesitzer,Jurist,Kammerrat,Kammerregistrator,Steuerassessor,NaN,NaN,NaN,NaN,NaN,None
5,https://database.factgrid.de/entity/Q1013,Freimaurerei,Illuminatenorden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1734-01-01,1805-01-01,None,None,Georg Friedrich,Schellhas,Prediger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,https://database.factgrid.de/entity/Q1014,Freimaurerei,Illuminatenorden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,N.N,Schelling,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,https://database.factgrid.de/entity/Q1016,Freimaurerei,Illuminatenorden,"Loge ""Ernst zum Compaß"", Gotha",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1753-01-01,1816-01-01,None,None,Christoph Friedrich Chrysostomus,Schenk,Rat der herzoglichen Kriegskanzlei,Schreiber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
8,https://database.factgrid.de/entity/Q1016,Freimaurerei,Illuminatenorden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rechtswissenschaften,1757-01-01,1805-01-01,None,None,Philipp Ernst,Scheppler,Amtsvogt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,https://database.factgrid.de/entity/Q1020,Freimaurerei,Illuminatenorden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,Jacob von,Scheurle,Hauptmann,NaN,NaN,NaN,NaN,NaN,NaN,

# Print all column titles to get an overview of the data structure

In [14]:
print(*df, sep="\n")

dc:identifier
gndo:affiliation_0_rdfs:literal
gndo:affiliation_1_rdfs:literal
gndo:affiliation_2_rdfs:literal
gndo:affiliation_3_rdfs:literal
gndo:affiliation_4_rdfs:literal
gndo:affiliation_5_rdfs:literal
gndo:affiliation_6_rdfs:literal
gndo:affiliation_7_rdfs:literal
gndo:affiliation_8_rdfs:literal
gndo:affiliation_9_rdfs:literal
gndo:biographicalOrHistoricalInformation_0_rdfs:literal
gndo:dateOfBirth_schema:date
gndo:dateOfDeath_schema:date
gndo:placeOfBirth_0_rdfs:literal
gndo:placeOfDeath_0_rdfs:literal
gndo:preferredNameOfThePerson_gndo:forename
gndo:preferredNameOfThePerson_gndo:surname
gndo:professionOrOccupation_0_rdfs:literal
gndo:professionOrOccupation_1_rdfs:literal
gndo:professionOrOccupation_2_rdfs:literal
gndo:professionOrOccupation_3_rdfs:literal
gndo:professionOrOccupation_4_rdfs:literal
gndo:professionOrOccupation_5_rdfs:literal
gndo:professionOrOccupation_6_rdfs:literal
gndo:professionOrOccupation_7_rdfs:literal
gndo:professionOrOccupation_8_rdfs:literal
gndo:profess

# Optional: generate ydata_profiling report

In [72]:
profile = ProfileReport(df, 
                        config_file="test_config.yaml", 
                        correlations=None,
                        interactions=None, 
                        missing_diagrams={
                            "heatmap": False,
                            "matrix": False,
                            },
                       )
profile.to_widgets()
#profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

# Generate detailed report for feedback 

In [15]:
my_df = ge.from_pandas(df)

#title of the report
report = "# Here are the validation results of your dataset:\n" 

#expectations based on header informations
report = report + "<br>"

result = my_df.expect_table_row_count_to_equal(number_in_set)
if (result['success']) == False:
    report = report + "<span style=\"color:red\"> **Row number does not match the size of the dataset.** </span>" + "<br>"
else:
    report = report + "<span style=\"color:green\">**Row number matches size of dataset.**</span>" + "<br>"

#expectations related to column titles
report = report + "<br>"

filter = [col for col in df if 'dc:identifier' in col]
if not filter:
    report = report + "<span style=\"color:red\">**Identifiers missing**</span>" + "<br>"
for i in filter:
    column_title = i
    report = report + "<span style=\"color:orange\">**Please check if column `" + column_title + "` contains identifiers from the source system.**</span>" + "<br>"
    
filter = [col for col in df if 'gndo:dateOfBirth' in col]
if not filter:
    report = report + "<span style=\"color:red\">**DateOfBirth is missing**</span>" + "<br>"
for i in filter:
    column_title = i
    report = report + "<span style=\"color:green\">**Column `" + column_title + "` with dateOfBirth specifications exists.**</span>" + "<br>"        
    
filter = [col for col in df if 'gndo:dateOfDeath' in col]
if not filter:
    report = report + "<span style=\"color:red\">**DateOfDeath is missing**</span>" + "<br>"
for i in filter:
    column_title = i
    report = report + "<span style=\"color:green\">**Column `" + column_title + "` with dateOfDeath specifications exists.**</span>" + "<br>"        
    
#expectations related to column content
report = report + "<br>"

filter = [col for col in df if 'dc:identifier' in col]
for i in filter:
    column_title = i
    result = my_df.expect_column_values_to_match_regex(column_title, "https\:\/\/database.factgrid.de\/entity\/Q(\d*|\d*X)$", result_format={"result_format": "COMPLETE"})
    failure_list = dict(zip(result['result']['unexpected_index_list'], result['result']['unexpected_list']))
    if (result['success']) == False:
        report = report + "<span style=\"color:red\">**Some values in column `" + column_title + "` are not URIs:**</span>" + "<br>"
        report = report + json.dumps(failure_list, indent="<br>") + "<br>"
    else:
        report = report + "<span style=\"color:green\">**Column `" + column_title + "` contains only URIs.**</span>" + "<br>"
    result = my_df.expect_column_values_to_be_unique(column_title, result_format={"result_format": "COMPLETE"})
    if (result['success']) == False:
        report = report + "<span style=\"color:red\">**Some values in column `" + column_title + "` are not unique:**</span>" + "<br>"
        failure_list = dict(zip(result['result']['unexpected_index_list'], result['result']['unexpected_list']))
        failure_list = dict(sorted(failure_list.items(), key=lambda item:item[1]))
        report = report + json.dumps(failure_list, indent="<br>") + "<br>"
    else:
        report = report + "<span style=\"color:green\">**Column `" + column_title + "` contains only unique values.**</span>"+ "<br>"

report = report + "<br>" 
        
filter = [col for col in df if 'preferred' in col and ('surname' in col or 'forename' in col)]
for i in filter:    
    column_title = i
    result = my_df.expect_column_values_to_not_be_null(column_title, result_format={"result_format": "COMPLETE"})
    failure_list = dict(zip(result['result']['unexpected_index_list'], result['result']['unexpected_list']))
    if (result['success']) == False:
        report = report + "<span style=\"color:orange\">**Some values are missing in column `" + column_title + "` .**</span>" + "<br>"
        report = report + json.dumps(failure_list, indent="<br>") + "<br>"
    else:
        report = report + "<span style=\"color:green\">**All rows in column `" + column_title + "` contain values.**</span>" + "<br>"     
    
report = report + "<br>"   

filter = [col for col in df if 'schema:date' in col]
for i in filter:
    column_title = i
    result = my_df.expect_column_values_to_match_regex_list(column_title, ["^\d{4}-\d{2}-\d{2}$", "^\d{4}$"], result_format={"result_format": "COMPLETE"})
    failure_list = dict(zip(result['result']['unexpected_index_list'], result['result']['unexpected_list']))
    if (result['success']) == False:
        report = report + "<span style=\"color:red\">**Some values in column `" + column_title + "` are not ISO 8601 YYYY-MM-DD compliant:**</span>" + "<br>"
        report = report + json.dumps(failure_list, indent="<br>") + "<br>"
    else:
        report = report + "<span style=\"color:green\">**Column `" + column_title + "` contains ISO 8601 YYYY-MM-DD compliant dates only.**</span>"+ "<br>"

display(Markdown(report))

# Here are the validation results of your dataset:
<br><span style="color:green">**Row number matches size of dataset.**</span><br><br><span style="color:orange">**Please check if column `dc:identifier` contains identifiers from the source system.**</span><br><span style="color:green">**Column `gndo:dateOfBirth_schema:date` with dateOfBirth specifications exists.**</span><br><span style="color:green">**Column `gndo:dateOfDeath_schema:date` with dateOfDeath specifications exists.**</span><br><br><span style="color:red">**Some values in column `dc:identifier` are not URIs:**</span><br>{
<br>"10": "test"
}<br><span style="color:red">**Some values in column `dc:identifier` are not unique:**</span><br>{
<br>"7": "https://database.factgrid.de/entity/Q1016",
<br>"8": "https://database.factgrid.de/entity/Q1016"
}<br><br><span style="color:orange">**Some values are missing in column `gndo:preferredNameOfThePerson_gndo:forename` .**</span><br>{
<br>"22": null,
<br>"28": null,
<br>"53": null,
<br>"108": null,
<br>"152": null,
<br>"158": null,
<br>"160": null,
<br>"185": null,
<br>"186": null,
<br>"187": null,
<br>"189": null,
<br>"196": null,
<br>"200": null,
<br>"206": null,
<br>"207": null,
<br>"211": null,
<br>"214": null,
<br>"222": null,
<br>"256": null,
<br>"273": null,
<br>"300": null,
<br>"330": null,
<br>"343": null,
<br>"344": null,
<br>"352": null,
<br>"360": null,
<br>"380": null,
<br>"429": null,
<br>"465": null,
<br>"522": null,
<br>"542": null,
<br>"556": null,
<br>"589": null,
<br>"627": null
}<br><span style="color:orange">**Some values are missing in column `gndo:preferredNameOfThePerson_gndo:surname` .**</span><br>{
<br>"185": null,
<br>"186": null,
<br>"187": null,
<br>"189": null,
<br>"200": null,
<br>"255": null,
<br>"267": null,
<br>"343": null
}<br><br><span style="color:red">**Some values in column `gndo:dateOfBirth_schema:date` are not ISO 8601 YYYY-MM-DD compliant:**</span><br>{
<br>"144": "1753-10-20|1754-10-20",
<br>"265": "1736-01-15|1738-01-01",
<br>"299": "1736-01-01|1737-01-01",
<br>"450": "1747-01-01|1748-12-17",
<br>"468": "1759-01-01|1762-01-01"
}<br><span style="color:red">**Some values in column `gndo:dateOfDeath_schema:date` are not ISO 8601 YYYY-MM-DD compliant:**</span><br>{
<br>"136": "1813-01-01|1813-03-27",
<br>"154": "1810-01-01|1811-11-20",
<br>"173": "1821-01-01|1825-01-01",
<br>"250": "1801-01-01|1847-11-18"
}<br>

# Write to md.file for PDF conversion

In [17]:
now = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

md_file = open('report_on_' + description + '_' + now + '.md', 'w+')
md_file.write(report)
md_file.close()